# 样本收集：


## 1. 爬虫


## 2. FFmpeg

- 样本采集车 ==> 路段 ==> 视频

- opencv 本质 ffmpeg
    - 具有音视频处理(编解码+API)的开源的软件库
    - 功能：
        - 文件格式和编解码的转换
        - 剪切 录制 提取 裁剪 复用
    - 获取视频的信息：
        - ffmpeg -i xxx.mp4
    - 视频到图片的分解：
        - ffmpeg -i xxx.mp4 image%d.jpg


In [2]:
import urllib
import urllib3
import os
from bs4 import BeautifulSoup

# 载入图片网站
url = "https://www.imooc.com"
# headers = ("User-Agent", "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36")
# opener = urllib.request.build_opener()
# opener.addheaders=[headers]
# html = opener.open(url)
html = urllib.request.urlopen(url).read()

# 解析图片
# 参数1：html，参数2：解析方法，参数3：编码类型
soup = BeautifulSoup(html, "html.parser", from_encoding="utf-8")

# 获取当前img标签
images = soup.findAll("img")
# print(images)

imageName = 0

for image in images:
    link = image.get("src")
#     print("link = ", link)
    link = "http:" + link
#     print("link = ", link)
    fileFormat = link[-3:]    # 获取当前link的最后三个字符
    if fileFormat == "jpg" or fileFormat == "png":
        fileSavePath = "/Users/zhengtaizhong/Desktop/T+O/crawler/" + str(imageName) + ".jpg"
        imageName = imageName +1
        try:
            urllib.request.urlretrieve(link, fileSavePath)    # 保存图片
        except Exception:
            pass
#             print("faile")
        

## 图像预处理 ==> 获取样本

In [7]:
import cv2

face_xml = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")
eye_xml = cv2.CascadeClassifier("./haarcascade_eye.xml")

image = cv2.imread("sinb.jpg",1)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# cv2.imshow("src", image)

# 人脸检测
faces = face_xml.detectMultiScale(gray, 1.2, 5)

print("face=", len(faces))
index = 0    # 要保存的图片名

# 给人脸画方框，(x,y) 人脸检测的起始坐标
for (x,y,w,h) in faces:
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    roi_face = gray[y:y+h, x:x+w]    # 定位人脸区域，检测人眼
    roi_color = image[y:y+h, x:x+w]
    file_path = "./roi/" + str(index)+ ".jpg"
    index = index + 1
    cv2.imwrite(file_path, roi_color)

# cv2.imshow("dst", image)
cv2.waitKey(5000)
cv2.destroyAllWindows()


face= 1


In [7]:
 # 1 数据yale 2 准备train label-》train # 3 cnn 4 检测
import tensorflow as tf
import numpy as np
import scipy.io as sio

# 15张人脸，每张人脸包含11张64x64图片，共165张图片，4097特征
data = open('Yale_64x64.mat','rb')
data_dict = sio.loadmat(data) 

train_data = data_dict['X']     # 加载训练数据
train_label = data_dict['Y']    # 加载训练标签


# 打乱训练：
# 训练图片的特征/标签值，无序排列
train_data = np.random.permutation(train_data)
train_label = np.random.permutation(train_label)

# 把一部分的训练数据作为测试数据。
test_data = train_data[0:64]
test_label = train_label[0:64]

# 测试图片的特征/标签值，无序排列（两次随机，保证键值对X-Y对齐）
np.random.seed(100)    # 设置随机种子
test_data = np.random.permutation(test_data)
np.random.seed(100)
test_label = np.random.permutation(test_label)


# (165, 4096) ==> (165,64,64,1), 转换数据类型，然后像素归一化[0,1]
train_data = train_data.reshape(train_data.shape[0],64,64,1).astype(np.float32)/255
# 构建165的训练标签
train_labels_new = np.zeros((165,15))    # 165张图片，15个人

for i in range(0,165):
    j = int(train_label[i,0])-1    # 1-15 ==> 0~14，j为train_label的下标
    train_labels_new[i,j] = 1      # 为什么是1 而不是 train_label[i,0]？

# 同上  
test_data_input = test_data.reshape(test_data.shape[0],64,64,1).astype(np.float32)/255
test_labels_input = np.zeros((64,15))# 165 image 15

for i in range(0,64):    # 64张测试图片
    j = int(test_label[i,0])-1 # 1-15 0-14 
    test_labels_input[i,j] = 1

data_input = tf.placeholder(tf.float32,[None,64,64,1])
label_input = tf.placeholder(tf.float32,[None,15])

layer1 = tf.layers.conv2d(inputs=data_input,filters=32,kernel_size=2,strides=1,padding='SAME',activation=tf.nn.relu)
layer1_pool = tf.layers.max_pooling2d(layer1,pool_size=2,strides=2)
layer2 = tf.reshape(layer1_pool,[-1,32*32*32])
layer2_relu = tf.layers.dense(layer2,1024,tf.nn.relu)
output = tf.layers.dense(layer2_relu,15)

loss = tf.losses.softmax_cross_entropy(onehot_labels=label_input,logits=output)
train = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
accuracy = tf.metrics.accuracy(labels=tf.argmax(label_input,axis=1),predictions=tf.argmax(output,axis=1))[1]
# run

# run acc
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
with tf.Session() as sess:
    sess.run(init)    
    for i in range(0,200):
        train_data_input = np.array(train_data)
        train_label_input = np.array(train_labels_new)
        sess.run([train,loss],feed_dict={data_input:train_data_input,label_input:train_label_input})
        acc = sess.run(accuracy,feed_dict={data_input:test_data_input,label_input:test_labels_input})
        print('acc:%.2f',acc)


acc:%.2f 0.046875
acc:%.2f 0.046875
acc:%.2f 0.046875
acc:%.2f 0.05078125
acc:%.2f 0.053125
acc:%.2f 0.059895832
acc:%.2f 0.06473214
acc:%.2f 0.068359375
acc:%.2f 0.07118055
acc:%.2f 0.0734375
acc:%.2f 0.07528409
acc:%.2f 0.078125
acc:%.2f 0.08173077
acc:%.2f 0.084821425
acc:%.2f 0.0875
acc:%.2f 0.08984375
acc:%.2f 0.09191176
acc:%.2f 0.09461805
acc:%.2f 0.097039476
acc:%.2f 0.09921875
acc:%.2f 0.10119048
acc:%.2f 0.10298295
acc:%.2f 0.105298914
acc:%.2f 0.107421875
acc:%.2f 0.109375
acc:%.2f 0.11177885
acc:%.2f 0.113425925


KeyboardInterrupt: 